In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r '/content/drive/Shareddrives/ML/MLP/train.zip' .
!cp -r '/content/drive/Shareddrives/ML/MLP/val.zip' .
!cp -r '/content/drive/Shareddrives/ML/MLP/test.zip' .
!unzip train.zip
!unzip val.zip
!unzip test.zip

Streaming output truncated to the last 5000 lines.
  inflating: test/Audi S4 Sedan 2007/01772.jpg  
  inflating: test/Audi S4 Sedan 2007/07660.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/05574.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/05670.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/06535.jpg  
  inflating: test/Audi S4 Sedan 2007/02743.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/05098.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/05814.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/04907.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/05078.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/02176.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/02831.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/03905.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/04396.jpg  
  inflating: test/Volkswagen Golf Hatchback 2012/04751.jpg  
  inflating: test/Audi S4 Sedan 2007/03217.jpg  
  inflating: test/Vol

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = 'train'
val_path = 'val'
test_path = 'test'

BATCH_SIZE = 64
train_datagen = ImageDataGenerator(rescale=1./255,
        # shear_range=0.2,
        # rotation_range=20.,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # zoom_range=[0.9, 1.25],
        # brightness_range=[0.5, 1.5],
        horizontal_flip=True).flow_from_directory(train_path, 
                                                  target_size=(224, 224), 
                                                  color_mode="rgb", 
                                                  batch_size=BATCH_SIZE, 
                                                  class_mode='categorical', 
                                                  shuffle=True, 
                                                  seed=1)
val_datagen = ImageDataGenerator(rescale=1./255,
        # shear_range=0.2,
        # rotation_range=20.,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # zoom_range=[0.9, 1.25],
        # brightness_range=[0.5, 1.5],
        horizontal_flip=True).flow_from_directory(val_path, 
                                                  target_size=(224, 224), 
                                                  color_mode="rgb", 
                                                  batch_size=BATCH_SIZE, 
                                                  class_mode='categorical', 
                                                  shuffle=True, 
                                                  seed=1)

test_datagen = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, 
                                                        target_size=(224, 224), 
                                                        color_mode="rgb", 
                                                        batch_size=BATCH_SIZE, 
                                                        class_mode='categorical',
                                                        shuffle=True, 
                                                        seed=1)

Found 12997 images belonging to 196 classes.
Found 1629 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [24]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model, optimizers, callbacks
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy

IMG_SIZE = 224
NUM_CLASSES = 196

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    # x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = Model(inputs, outputs, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=[CategoricalAccuracy(), TopKCategoricalAccuracy(k=3)]
    )
    return model

In [25]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=[CategoricalAccuracy(), TopKCategoricalAccuracy(k=3)]
    )

model = build_model(num_classes=NUM_CLASSES)

earlystopping = callbacks.EarlyStopping(monitor = 'val_categorical_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = callbacks.ModelCheckpoint(filepath = '/content/drive/Shareddrives/ML/efficientnet',
                                                  monitor = 'val_categorical_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  mdome = 'max')
lr_reduce = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=1e-5)

callbacks = [earlystopping, checkpointer, lr_reduce]

STEP_SIZE_TRAIN = train_datagen.n // train_datagen.batch_size
STEP_SIZE_VALID = val_datagen.n // val_datagen.batch_size

epochs = 30
history = model.fit(x=train_datagen, epochs=epochs, validation_data=val_datagen, steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID, callbacks=callbacks)

Epoch 1/30
 43/203 [=====>........................] - ETA: 1:03 - loss: 5.7470 - categorical_accuracy: 0.0025 - top_k_categorical_accuracy: 0.0124

KeyboardInterrupt: ignored